# LogReg

### Import

In [ ]:
from numpy import dot
from pandas import read_csv

## Učitavanje dataseta

Funkcija za učitavanje podataka iz CSV datoteke

In [ ]:
def df(filename):
    return read_csv(filename)

### Train set

In [ ]:
train = df(open("500train_BoW.csv"))

Odvajanje targeta od ulaznih vektora prikazqnih Bag-of-Words modelom. Targete skaliramo sa $\{1, 2\}$ na $\{0, 1\}$, gdje 0 predstavlja loš review, a 1 dobar review.

In [ ]:
inputVektori = []
inputTargeti = []
iterator = train.iterrows()
for i in range(train.shape[0]):
    # whole_row = next(iterator)
    row = next(iterator)[1]
    inputTargeti.append(row["TARGET"] - 1)
    inputVektori.append(row[1:])

print(inputVektori)
print(inputTargeti)

## Test set

In [ ]:
test = df(open("20test_BoW.csv"))

testVektori = []
testTargeti = []
iterator = test.iterrows()
for i in range(test.shape[0]):
    row = next(iterator)[1]
    testTargeti.append(row["TARGET"] - 1)
    testVektori.append(row[1:])

print(testVektori)
print(testTargeti)

## Definicije funkcija za logističku regresiju

- Sigma 
- ForwardPass
- BackProp

### Sigma

$$ \sigma (z) = \frac{1}{1 + e^{-z}} $$

pri čemu $e$ aproksimiramo s $2.71$.

In [ ]:
def Sigma(z):
    return 1 / ( 1 + (2.71**(-z)) )

### ForwardPass

Varijable outputs i errors popunjavamo pri svakom pozivu ForwardPassa. Rezultat računamo iz umnoška (funkcija dot iz numpya) ulaznog vektora i težina:

$$ \hat{y} = \sigma (w \times x) $$

Greška:

$$ E = \frac{1}{2} (t - \hat{y})^{2} $$


In [ ]:
outputs = []
errors = []

def ForwardPass(tezine, inputVektor, target):
    y_kapica = Sigma(dot(tezine, inputVektor))
    greska = 0.5 * ((target - y_kapica)**2)
    outputs.append(y_kapica)
    errors.append(greska)
    return (y_kapica, greska)

### BackProp

Definiramo brzinu učenja kao $ \eta = 0.25 $, umjesto predloženih $0.1$ zbog relativno malog broja epoha.

Ažuriramo težine na temelju rezultata i targeta:

In [ ]:
eta = 0.25

def BackProp(tezine, inputVektor, y_kapica, target):
    noveTezine = []
    for i in range(len(tezine)):
        trenutnaTezina = tezine[i]
        parDer = - inputVektor[i] * y_kapica * (1 - y_kapica) * (target - y_kapica)
        novaTezina = trenutnaTezina - eta * parDer
        noveTezine.append(novaTezina)
    return noveTezine

## Učenje

U svakoj od $N = 25$ epoha izvedemo ForwardPass i BackProp za sve ulazne vektore.

In [ ]:
brojRijeci = test.shape[1] - 1
brojReviewova = train.shape[0]

def Train(trainSet, weights, targets, N=2):
    tezinePoEpohama = [weights]
    for epoha in range(N):
        for i in range(brojReviewova):
            y_kapica, greska = ForwardPass(tezinePoEpohama[-1], trainSet[i], targets[i])
            tezinePoEpohama.append(BackProp(tezinePoEpohama[-1], trainSet[i], y_kapica, targets[i]))
    return tezinePoEpohama

pocetneTezine = []
for i in range(brojRijeci):
    pocetneTezine.append(0.5)

tezinePoEpohama = Train(inputVektori, pocetneTezine, inputTargeti, N=25)
konacneTezine = tezinePoEpohama[-1]

for i in range(len(inputVektori[0].keys())):
    print(inputVektori[0].keys()[i], konacneTezine[i])

## Testiranje

Na testnom skupu podataka evaluiramo težine koje smo dobili učenjem.

In [ ]:
def Predict(tezine, inputVektori):
    predikcije = []
    for i in range(len(inputVektori)):
        y_kapica = Sigma(dot(tezine, inputVektori[i]))
        predikcije.append(y_kapica)
    return predikcije

predikcije = Predict(konacneTezine, testVektori)

print(" Pred | Target")
print("------+--------")
for i in range(len(predikcije)):
    print(" {:.2f} |   {}".format(predikcije[i], testTargeti[i]))